In [ ]:
import pandas as pd
import requests
import io
import numpy as np  
from datetime import date, timedelta
import re
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
#from github import Github
#import github
import torch
import torch.nn as nn
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader
# Import nn.functional
import torch.nn.functional as F
import torch.optim as optim
from typing import Union, Tuple
import os
import sys
import time
from collections import OrderedDict
from sklearn.preprocessing import MinMaxScaler
from statistics import mean
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import math
import random
import imageio
#from sklearn.metrics import mean_absolute_percentage_error
matplotlib.style.use('seaborn')
%matplotlib inline
#random.seed(42)
#torch.manual_seed(42)
#np.random.seed(42)

In [ ]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        weight = next(self.parameters()).data
        h = weight.new(self.n_layers, x.size(0), self.hidden_dim).zero_()
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out

In [ ]:
Shortlisted_States=['Maharashtra','Delhi','Uttar-Pradesh','Kerala','Tamil-Nadu']
results_gru=[]
for state in Shortlisted_States:
  best_models=[]
  df=pd.read_csv("https://raw.githubusercontent.com/sureshkuc/Data-Science-in-Life-Science-Project/main/Indian-States-Covid19-Datasets/"+state+".csv", parse_dates=["Date"]).drop(columns =["Unnamed: 0"])
  df = df[df["Date"] > "2020-03-19"]
  df = df.set_index("Date")
  df = df[['Confirmed', 'Recovered', 'Deceased', 'New_Confirmerd', 'New_Deaths', 'New_Recovered']]
  #print(df.describe())

  time_step=[5,7,15]
  Number_of_feature=[1,2,3,4,5,6]
  multi_feature=True
  output_dim=1
  for n_f in Number_of_feature:
    for t_s in time_step:
      train_loader, test_loader = data_preparation(df, scaling_range=(0,1),time_step=t_s,number_feature=n_f, response_variable_index=0,data_split_ratio=0.8, Suffle=False)
      for n_layers in range(1,3,1):
        for n_hidden_nodes in [1,5,8,16,32]:
          
          max_epochs=25
          random.seed(42)
          torch.manual_seed(42)
          np.random.seed(42)
          #CNN model with L1 loss
          #best_model=Call_CNN_model(state,dataset=(train_loader, test_loader), lr=1e-2,criterion=nn.L1Loss(),max_epochs=max_epochs)
          GRUNet_model = GRUNet(n_f, n_hidden_nodes, output_dim, n_layers)
          #if torch.cuda.is_available():
          #stm_model = lstm_model.cuda()
          #gru_optim = optim.SGD(GRUNet_model.parameters(), lr=1e-3, momentum=0.9)
          gru_optim = optim.Adam(GRUNet_model.parameters(), lr=1e-3)
          train_losses,test_losses,best_model = fit(GRUNet_model, gru_optim,nn.L1Loss(),(train_loader, test_loader), max_epochs=max_epochs,cuda=False)
          #print(f'\nTraining took {end-start}s!')
          #plot_loss(max_epochs,train_losses,test_losses,model_name='CNN for '+state)
          GRUNet_model = GRUNet(n_f, n_hidden_nodes, output_dim, n_layers)
          GRUNet_model.load_state_dict(best_model)
          GRUNet_model.eval()
          test_x,test_y=test_loader
          predictions=GRUNet_model(test_x)
          test_y=test_y.cpu().detach().numpy()
          predictions=predictions.cpu().detach().numpy()
          mae=mean_absolute_error(test_y,predictions)
          rmse=math.sqrt(mean_squared_error(test_y,predictions))
          #mape=mean_absolute_percentage_error(test_y,predictions)
          r2s=r2_score(test_y,predictions)
          results_gru.append([state,n_f,t_s,n_layers,n_hidden_nodes,mae,rmse,r2s])
          print(state,'n_f',n_f,'t_s',t_s,'n_layers',n_layers,n_hidden_nodes,'Error',mae,rmse,r2s)